In [ ]:
import polars as pl
import polars_ds as pds

In [ ]:
df = pds.random_data(size=5_000, n_cols=0).select(
    pds.random(0.0, 1.0).alias("x1"),
    pds.random(0.0, 1.0).alias("x2"),
    pds.random(0.0, 1.0).alias("x3"),
).with_columns(
    y = pl.col("x1") * 0.5 + pl.col("x2") * 0.25 - pl.col("x3") * 0.15 + pds.random() * 0.0001
)

In [ ]:

df.select(
    pds.query_lstsq(
        "x1", "x2", "x3", 
        target = "y",
        method = "l2",
        l2_reg = 0.05,
        add_bias = True
    ).alias("coeffs")
).explode("coeffs") # .item(0,0)

In [ ]:
from sklearn import linear_model
from scipy.linalg import lstsq
df_pd = df.to_pandas()
x = df_pd[["x1", "x2", "x3"]]
y = df_pd["y"]

In [ ]:

reg = linear_model.Ridge(alpha = 0.05, fit_intercept=True)
reg.fit(x, y)

In [ ]:
print(reg.coef_, reg.intercept_)

In [ ]:
%%timeit
reg = linear_model.LinearRegression(fit_intercept=True)
reg.fit(x, y)

In [ ]:
print(reg.coef_, reg.intercept_)

In [ ]:
reg = linear_model.Ridge(alpha=.1)
reg.fit(x, y)

In [ ]:
%%timeit
reg = linear_model.Ridge(alpha=.1)
reg.fit(x, y)

In [ ]:
reg.coef_